In [1]:
from htmd.ui import *
config(viewer='ngl')
#os.chdir('/webdata/73hboiwia98hdj209jq0/')  # Skip this command.


Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049

You are on the latest HTMD version (unpackaged : /home/joao/maindisk/software/repos/Acellera/htmd/htmd).



# System Building μ-opioid Receptor in Membrane

## Prepare the protein

(Download the required files for the tutorial from this [link](http://pub.htmd.org/73hboiwia98hdj209jq0/mor.tar.gz).)

View the file as it comes from the OPM database.

In [2]:
Molecule('mor/4dkl.pdb').view()

Retrieve and clean up `DUM` marker atoms.

In [3]:
from htmd.util import opm
prot, thickness = opm('4dkl')
thickness

2017-01-10 23:20:06,384 - htmd.molecule.molecule - INFO - Removed 2546 atoms. 4836 atoms remaining in the molecule.


32.0

Remove non-protein atoms, and keep only a monomer.

In [4]:
prot.filter('protein and noh and chain B or water within 5 of (chain B and protein)')

2017-01-10 23:20:10,482 - htmd.molecule.molecule - INFO - Removed 2574 atoms. 2262 atoms remaining in the molecule.


Automatically detecting segments and assigning names to them.

In [5]:
prot = autoSegment(prot,'protein')

2017-01-10 23:20:14,499 - htmd.builder.builder - INFO - Created segment P0 between resid 65 and 263.
2017-01-10 23:20:14,501 - htmd.builder.builder - INFO - Created segment P1 between resid 270 and 352.


## Build the protein

In [6]:
topos  = charmm.defaultTopo() + ['mor/ff.rtf']
params = charmm.defaultParam() + ['mor/ff.prm']

prot = charmm.build(prot, topo=topos, param=params, 
                    outdir='./03c_out_morbuild/prot/', ionize=False)

2017-01-10 23:20:19,379 - htmd.builder.charmm - INFO - Writing out segments.


Bond between A: [serial 3005 resid 140 resname CYS chain B segid P0]
             B: [serial 3615 resid 217 resname CYS chain B segid P0]



2017-01-10 23:20:20,584 - htmd.builder.builder - INFO - One disulfide bond was added
2017-01-10 23:20:20,753 - htmd.builder.charmm - INFO - Starting the build.
2017-01-10 23:20:20,865 - htmd.builder.charmm - WARNING - Failed to guess coordinates for 1 atoms due to bad angles.
2017-01-10 23:20:20,867 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 78 atoms.
2017-01-10 23:20:20,868 - htmd.builder.charmm - WARNING - Please check /data/joao/maindisk/SANDBOX/03_PreparationBuilding_TGiorgino/03c_out_morbuild/prot/log.txt for further information.
2017-01-10 23:20:20,869 - htmd.builder.charmm - INFO - Finished building.


In [7]:
prot.reps.add(sel='segid P0', style='NewCartoon', color=1)
prot.reps.add(sel='segid P1', style='NewCartoon', color=2)
prot.view()

## Add a sodium atom in the receptor

In [8]:
sod = Molecule('mor/sod.pdb')
sod.set('segid','S1')
prot.append(sod)
prot.reps.add(sel='ions', style='VDW')
prot.view()

## Embed the protein into a membrane

In [9]:
memb = Molecule('mor/membrane80by80C36.pdb')

Center the membrane onto the protein center

In [10]:
pcenter = np.mean(prot.get('coords','protein'),axis=0)
mcenter = np.mean(memb.get('coords'),axis=0)
memb.moveBy(pcenter-mcenter)

And now embed. 

The two are equivalent - `append` with `collisions=True` only
adds atoms if they do not clash  sterically.

In [11]:
mol = prot.copy()
mol.append(memb, collisions=True)
# mol = embed(prot,memb)

2017-01-10 23:20:51,502 - htmd.molecule.molecule - INFO - Removed 305 residues from appended Molecule due to collisions.


## Visualize the embedded system

In [12]:
mol.reps.add(sel='protein', style='NewCartoon', color='Secondary Structure')
mol.reps.add(sel='ions', style='VDW')
mol.reps.add(sel='lipids', style='Lines')
mol.view()

## Add a ligand

In [13]:
lig = Molecule('mor/QM-min.pdb') 
lig.set('segid','L');
lcenter = np.mean(lig.get('coords'),axis=0)
newlcenter=[random.uniform(-10, 10), random.uniform(-10, 10),  43 ]
lig.rotateBy(uniformRandomRotation(), lcenter)
lig.moveBy(newlcenter-lcenter)
mol.append(lig)

## Put it in a water box

In [14]:
coo = mol.get('coords','noh and (lipids or protein)')
m = np.min(coo, axis=0) + [0, 0, -5]
M = np.max(coo, axis=0) + [0, 0, 20]
smol = solvate(mol, minmax=np.vstack((m,M)))

2017-01-10 23:35:27,474 - htmd.builder.solvate - INFO - Using water pdb file at: /data/joao/maindisk/software/repos/Acellera/htmd/htmd/builder/wat.pdb
2017-01-10 23:35:28,940 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2


Solvating: 100% (8/8) [############################################] eta 00:00 /


2017-01-10 23:36:06,230 - htmd.builder.solvate - INFO - After removing water molecules colliding with other molecules, 9116 water molecules were added to the system.


## Visualize

In [15]:
smol.reps.add(sel='segid L', style='Licorice')
smol.reps.add(sel='water', style='Lines')
smol.view()

## Build with CHARMM

In [16]:
molbuilt = charmm.build(smol, topo=topos, param=params, 
                        outdir='./build/',
                        saltconc=0.15)

2017-01-10 23:36:50,359 - htmd.builder.charmm - INFO - Writing out segments.


Bond between A: [serial 1212 resid 140 resname CYS chain  segid P0]
             B: [serial 2448 resid 217 resname CYS chain  segid P0]



2017-01-10 23:38:48,651 - htmd.builder.builder - INFO - One disulfide bond was added
2017-01-10 23:38:48,830 - htmd.builder.charmm - INFO - Starting the build.
2017-01-10 23:38:49,628 - htmd.builder.charmm - INFO - Finished building.
2017-01-10 23:38:53,686 - htmd.builder.ionize - INFO - Adding 14 anions + 0 cations for neutralizing and 66 ions for the given salt concentration.
2017-01-10 23:38:54,202 - htmd.builder.ionize - INFO - Min distance of ions from molecule: 5A
2017-01-10 23:38:54,203 - htmd.builder.ionize - INFO - Min distance between ions: 5A
2017-01-10 23:38:54,204 - htmd.builder.ionize - INFO - Placing 80 ions.
2017-01-10 23:39:48,897 - htmd.builder.charmm - INFO - Writing out segments.
2017-01-10 23:41:40,310 - htmd.builder.charmm - INFO - Starting the build.
2017-01-10 23:41:41,095 - htmd.builder.charmm - INFO - Finished building.


## Visualize built system

In [17]:
molbuilt.view()